In [28]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv('train_for_progosis.csv')

In [19]:
df

,patient_id,surgery_time,last_follow,state_of_survival,diff,months,censor,slide_id
0,1602191,2016-01-20,2021-09-01,1,2051 days,67.38,0,1602191.svs
1,1604538,2016-02-23,2016-11-10,2,261 days,8.57,0,1604538.svs
2,1604535,2016-02-23,2016-05-03,2,70 days,2.30,0,1604535.svs
3,1604941,2016-02-25,2021-07-05,1,1957 days,64.29,0,1604941.svs
4,1605354,2016-02-29,2022-09-02,1,2377 days,78.09,0,1605354.svs
...,...,...,...,...,...,...,...,...
264,1847271,2018-11-28,2022-09-02,1,1374 days,45.14,0,1847271.svs
265,1904182,2019-01-25,2021-02-14,2,751 days,24.67,0,1904182.svs
266,1934300,2019-08-02,2022-05-03,2,1005 days,33.02,0,1934300.svs
267,1935165,2019-08-08,2020-08-24,2,382 days,12.55,0,1935165.svs


In [18]:
df['slide_id'] = [str(i)+'.svs' for i in df['patient_id']]

In [25]:
label_col = 'months'
n_bins = 4
eps = 1e-6

In [29]:
def add_bins(slide_data):
    assert 'patient_id' in slide_data.columns and 'censor' in slide_data.columns
    
    patients_df = slide_data.drop_duplicates(['patient_id']).copy()
    uncensored_df = patients_df[patients_df['censor'] < 1]
    disc_labels, q_bins = pd.qcut(uncensored_df[label_col], q=n_bins, retbins=True, labels=False)
    q_bins[-1] = slide_data[label_col].max() + eps
    q_bins[0] = slide_data[label_col].min() - eps

    disc_labels, q_bins = pd.cut(patients_df[label_col], bins=q_bins, retbins=True, labels=False, right=False, include_lowest=True)
    patients_df.insert(2, 'label', disc_labels.values.astype(int))

    patient_dict = {}
    slide_data = slide_data.set_index('patient_id')
    for patient in patients_df['patient_id']:
        slide_ids = slide_data.loc[patient, 'slide_id']
        if isinstance(slide_ids, str):
            slide_ids = np.array(slide_ids).reshape(-1)
        else:
            slide_ids = slide_ids.values
        patient_dict.update({patient:slide_ids})
        
    return q_bins, patient_dict, patients_df

In [30]:
slide_data = df

In [31]:
q_bins, patients_dict, slide_data = add_bins(slide_data)

In [37]:
slide_data.to_csv('train_for_progosis_268.csv',index=None)

In [38]:
df = pd.read_csv('patient_50.csv')

In [44]:
df.rename(columns={'patient':'patient_id'}, inplace=True)

In [45]:
df

,path,patient_id
0,300_tumor/352990_tile_10779_level0_8481-18663-...,352990
1,300_tumor/352990_tile_49363_level0_35681-17703...,352990
2,300_tumor/352990_tile_53202_level0_38401-14503...,352990
3,300_tumor/352990_tile_42494_level0_30881-8263-...,352990
4,300_tumor/352990_tile_10288_level0_8161-12743-...,352990
...,...,...
12145,300_tumor/1643350_tile_26180_level0_19180-3342...,1643350
12146,300_tumor/1643350_tile_27734_level0_20300-3342...,1643350
12147,300_tumor/1643350_tile_27285_level0_19980-3262...,1643350
12148,300_tumor/1643350_tile_20175_level0_14860-3166...,1643350


In [157]:
new_df = pd.read_excel('HMU-GC队列.xlsx')

In [60]:
df['patient_id'].value_counts()

patient_id
352990     50
712331     50
699616     50
699673     50
699831     50
           ..
664041     50
664256     50
665864     50
665898     50
1643350    50
Name: count, Length: 243, dtype: int64

In [70]:
699616 in new_df['病理号']

False

In [68]:
665898 in new_df['住院号']

False

In [66]:
new_df['住院号'] = [int(i) for i in new_df['住院号']]

In [98]:
new_df['病理号'] = [int(i) for i in new_df['病理号']]

In [74]:
1643350 in new_df['病理号']

False

In [76]:
list1 = slide_data['patient_id']

In [99]:
list2 = new_df['病理号']

In [102]:
new_df['patient_id'] = [int(i) for i in new_df['病理号']]

In [106]:
type(new_df['patient_id'][1])

numpy.int64

In [113]:
len([i for i in list1 for j in list2 if i==j])

269

In [122]:
df['patient_id'].drop_duplicates()

0         352990
50        561635
100       578002
150       585570
200       624460
          ...   
11900    1625423
11950    1634403
12000    1634929
12050    1638800
12100    1643350
Name: patient_id, Length: 243, dtype: int64

In [115]:
slide_data

,patient_id,surgery_time,label,last_follow,state_of_survival,diff,months,censor,slide_id
0,1602191,2016-01-20,3,2021-09-01,1,2051 days,67.38,0,1602191.svs
1,1604538,2016-02-23,0,2016-11-10,2,261 days,8.57,0,1604538.svs
2,1604535,2016-02-23,0,2016-05-03,2,70 days,2.30,0,1604535.svs
3,1604941,2016-02-25,3,2021-07-05,1,1957 days,64.29,0,1604941.svs
4,1605354,2016-02-29,3,2022-09-02,1,2377 days,78.09,0,1605354.svs
...,...,...,...,...,...,...,...,...,...
264,1847271,2018-11-28,2,2022-09-02,1,1374 days,45.14,0,1847271.svs
265,1904182,2019-01-25,1,2021-02-14,2,751 days,24.67,0,1904182.svs
266,1934300,2019-08-02,1,2022-05-03,2,1005 days,33.02,0,1934300.svs
267,1935165,2019-08-08,1,2020-08-24,2,382 days,12.55,0,1935165.svs


In [137]:
df.rename(columns={'patient_id':'merge'},inplace=
         True)

In [148]:
x = df.merge(slide_data,on='merge',how='outer')

In [135]:
slide_data

,patient_id,surgery_time,label,last_follow,state_of_survival,diff,months,censor,slide_id
0,1602191,2016-01-20,3,2021-09-01,1,2051 days,67.38,0,1602191.svs
1,1604538,2016-02-23,0,2016-11-10,2,261 days,8.57,0,1604538.svs
2,1604535,2016-02-23,0,2016-05-03,2,70 days,2.30,0,1604535.svs
3,1604941,2016-02-25,3,2021-07-05,1,1957 days,64.29,0,1604941.svs
4,1605354,2016-02-29,3,2022-09-02,1,2377 days,78.09,0,1605354.svs
...,...,...,...,...,...,...,...,...,...
264,1847271,2018-11-28,2,2022-09-02,1,1374 days,45.14,0,1847271.svs
265,1904182,2019-01-25,1,2021-02-14,2,751 days,24.67,0,1904182.svs
266,1934300,2019-08-02,1,2022-05-03,2,1005 days,33.02,0,1934300.svs
267,1935165,2019-08-08,1,2020-08-24,2,382 days,12.55,0,1935165.svs


In [144]:
df

,path,merge
0,300_tumor/352990_tile_10779_level0_8481-18663-...,352990
1,300_tumor/352990_tile_49363_level0_35681-17703...,352990
2,300_tumor/352990_tile_53202_level0_38401-14503...,352990
3,300_tumor/352990_tile_42494_level0_30881-8263-...,352990
4,300_tumor/352990_tile_10288_level0_8161-12743-...,352990
...,...,...
12145,300_tumor/1643350_tile_26180_level0_19180-3342...,1643350
12146,300_tumor/1643350_tile_27734_level0_20300-3342...,1643350
12147,300_tumor/1643350_tile_27285_level0_19980-3262...,1643350
12148,300_tumor/1643350_tile_20175_level0_14860-3166...,1643350


In [145]:
slide_data

,merge,surgery_time,label,last_follow,state_of_survival,diff,months,censor,slide_id
0,1602191,2016-01-20,3,2021-09-01,1,2051 days,67.38,0,1602191.svs
1,1604538,2016-02-23,0,2016-11-10,2,261 days,8.57,0,1604538.svs
2,1604535,2016-02-23,0,2016-05-03,2,70 days,2.30,0,1604535.svs
3,1604941,2016-02-25,3,2021-07-05,1,1957 days,64.29,0,1604941.svs
4,1605354,2016-02-29,3,2022-09-02,1,2377 days,78.09,0,1605354.svs
...,...,...,...,...,...,...,...,...,...
264,1847271,2018-11-28,2,2022-09-02,1,1374 days,45.14,0,1847271.svs
265,1904182,2019-01-25,1,2021-02-14,2,751 days,24.67,0,1904182.svs
266,1934300,2019-08-02,1,2022-05-03,2,1005 days,33.02,0,1934300.svs
267,1935165,2019-08-08,1,2020-08-24,2,382 days,12.55,0,1935165.svs


In [146]:
df, slide_data

(                                                    path    merge
 0      300_tumor/352990_tile_10779_level0_8481-18663-...   352990
 1      300_tumor/352990_tile_49363_level0_35681-17703...   352990
 2      300_tumor/352990_tile_53202_level0_38401-14503...   352990
 3      300_tumor/352990_tile_42494_level0_30881-8263-...   352990
 4      300_tumor/352990_tile_10288_level0_8161-12743-...   352990
 ...                                                  ...      ...
 12145  300_tumor/1643350_tile_26180_level0_19180-3342...  1643350
 12146  300_tumor/1643350_tile_27734_level0_20300-3342...  1643350
 12147  300_tumor/1643350_tile_27285_level0_19980-3262...  1643350
 12148  300_tumor/1643350_tile_20175_level0_14860-3166...  1643350
 12149  300_tumor/1643350_tile_24391_level0_17900-3134...  1643350
 
 [12150 rows x 2 columns],
        merge surgery_time  label last_follow  state_of_survival       diff  \
 0    1602191   2016-01-20      3  2021-09-01                  1  2051 days   
 1    160

In [149]:
x

,path,merge,surgery_time,label,last_follow,state_of_survival,diff,months,censor,slide_id
0,300_tumor/352990_tile_10779_level0_8481-18663-...,352990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,300_tumor/352990_tile_49363_level0_35681-17703...,352990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,300_tumor/352990_tile_53202_level0_38401-14503...,352990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,300_tumor/352990_tile_42494_level0_30881-8263-...,352990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,300_tumor/352990_tile_10288_level0_8161-12743-...,352990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12414,NaN,1847271,2018-11-28,2.0,2022-09-02,1.0,1374 days,45.14,0.0,1847271.svs
12415,NaN,1904182,2019-01-25,1.0,2021-02-14,2.0,751 days,24.67,0.0,1904182.svs
12416,NaN,1934300,2019-08-02,1.0,2022-05-03,2.0,1005 days,33.02,0.0,1934300.svs
12417,NaN,1935165,2019-08-08,1.0,2020-08-24,2.0,382 days,12.55,0.0,1935165.svs


In [150]:
slide_data

,merge,surgery_time,label,last_follow,state_of_survival,diff,months,censor,slide_id
0,1602191,2016-01-20,3,2021-09-01,1,2051 days,67.38,0,1602191.svs
1,1604538,2016-02-23,0,2016-11-10,2,261 days,8.57,0,1604538.svs
2,1604535,2016-02-23,0,2016-05-03,2,70 days,2.30,0,1604535.svs
3,1604941,2016-02-25,3,2021-07-05,1,1957 days,64.29,0,1604941.svs
4,1605354,2016-02-29,3,2022-09-02,1,2377 days,78.09,0,1605354.svs
...,...,...,...,...,...,...,...,...,...
264,1847271,2018-11-28,2,2022-09-02,1,1374 days,45.14,0,1847271.svs
265,1904182,2019-01-25,1,2021-02-14,2,751 days,24.67,0,1904182.svs
266,1934300,2019-08-02,1,2022-05-03,2,1005 days,33.02,0,1934300.svs
267,1935165,2019-08-08,1,2020-08-24,2,382 days,12.55,0,1935165.svs


In [153]:
df['merge'].drop_duplicates()

0         352990
50        561635
100       578002
150       585570
200       624460
          ...   
11900    1625423
11950    1634403
12000    1634929
12050    1638800
12100    1643350
Name: merge, Length: 243, dtype: int64

In [156]:
df

,path,merge
0,300_tumor/352990_tile_10779_level0_8481-18663-...,352990
1,300_tumor/352990_tile_49363_level0_35681-17703...,352990
2,300_tumor/352990_tile_53202_level0_38401-14503...,352990
3,300_tumor/352990_tile_42494_level0_30881-8263-...,352990
4,300_tumor/352990_tile_10288_level0_8161-12743-...,352990
...,...,...
12145,300_tumor/1643350_tile_26180_level0_19180-3342...,1643350
12146,300_tumor/1643350_tile_27734_level0_20300-3342...,1643350
12147,300_tumor/1643350_tile_27285_level0_19980-3262...,1643350
12148,300_tumor/1643350_tile_20175_level0_14860-3166...,1643350


In [160]:
new_df = pd.DataFrame({'merge':new_df['病理号'], 'patient_id':new_df['住院号']})

In [163]:
slide_data = pd.merge(new_df,slide_data)

In [164]:
df['patient_id'] = df['merge']

In [167]:
df = pd.merge(df,slide_data,on='patient_id')

In [170]:
del df['merge_y']

In [173]:
del df['merge_x']

In [177]:
df.to_csv('data_train_for_progosis_209.csv',index=None)

In [175]:
df['patient_id'].drop_duplicates()

0        352990
50       561635
100      578002
150      585570
200      624460
          ...  
10200    754334
10250    761887
10300    786348
10350    790354
10400    794635
Name: patient_id, Length: 209, dtype: int64

In [178]:
df

,path,patient_id,surgery_time,label,last_follow,state_of_survival,diff,months,censor,slide_id
0,300_tumor/352990_tile_10779_level0_8481-18663-...,352990,2016-09-27,0,2017-05-11,2,226 days,7.42,0,1633139.svs
1,300_tumor/352990_tile_49363_level0_35681-17703...,352990,2016-09-27,0,2017-05-11,2,226 days,7.42,0,1633139.svs
2,300_tumor/352990_tile_53202_level0_38401-14503...,352990,2016-09-27,0,2017-05-11,2,226 days,7.42,0,1633139.svs
3,300_tumor/352990_tile_42494_level0_30881-8263-...,352990,2016-09-27,0,2017-05-11,2,226 days,7.42,0,1633139.svs
4,300_tumor/352990_tile_10288_level0_8161-12743-...,352990,2016-09-27,0,2017-05-11,2,226 days,7.42,0,1633139.svs
...,...,...,...,...,...,...,...,...,...,...
10445,300_tumor/794635_tile_64887_level0_38422-34081...,794635,2019-09-12,0,2019-11-22,2,71 days,2.33,0,1940937.svs
10446,300_tumor/794635_tile_21090_level0_13462-15361...,794635,2019-09-12,0,2019-11-22,2,71 days,2.33,0,1940937.svs
10447,300_tumor/794635_tile_51966_level0_31062-27521...,794635,2019-09-12,0,2019-11-22,2,71 days,2.33,0,1940937.svs
10448,300_tumor/794635_tile_43306_level0_26102-30721...,794635,2019-09-12,0,2019-11-22,2,71 days,2.33,0,1940937.svs


In [179]:
df['label'].value_counts()

label
1    3050
2    2550
0    2450
3    2400
Name: count, dtype: int64